In [1]:
# python -m pip install -e .

In [2]:
import pandas as pd
import spacy_annotator as spa

## Example

In [3]:
df = pd.DataFrame({
    "text": [
        "New york is lovely, Milan is nice, but london is amazing!",
        "Stockholm is too cold. Ingrid Bergman says so."
    ]})
df["index"] = df.index

In [4]:
df

,text,index
0,"New york is lovely, Milan is nice, but london ...",0
1,Stockholm is too cold. Ingrid Bergman says so.,1


### Annotation without model

In [5]:
annotator = spa.Annotator(labels=["GPE", "PERSON"])

In [6]:
annotator.instructions


            Instructions 

            For each entity type, input must be a DELIMITER separated string. 

            If no entities in text, leave as is and press submit.
            Similarly, if no entities for a particular label, leave as is. 

            Buttons: 

            	 * submit inserts new annotation (or overwrites existing one if one is present). 

            	 * skip moves forward and leaves empty string (or existing annotation if one is present). 

            	 * finish terminates the annotation session.
            


In [7]:
df_labels = annotator.annotate(df=df, col_text="text")

HTML(value='-1 examples annotated, 3 examples left')

Text(value='', description='GPE', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='PERSON', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Output()

In [8]:
df_labels

,text,index,annotations
0,"New york is lovely, Milan is nice, but london ...",0,
1,Stockholm is too cold. Ingrid Bergman says so.,1,


In [9]:
df_labels.annotations[1]

''

### Annotation with model

In [10]:
import spacy

In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
annotator = spa.Annotator(labels=["GPE", "PERSON"], model=nlp)

In [13]:
df_labels = annotator.annotate(df=df, col_text="text", shuffle=True)

HTML(value='-1 examples annotated, 3 examples left')

Text(value='', description='GPE', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='PERSON', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Output()

In [14]:
df_labels

,text,index,annotations
0,"New york is lovely, Milan is nice, but london ...",0,"(New york is lovely, Milan is nice, but london..."
1,Stockholm is too cold. Ingrid Bergman says so.,1,(Stockholm is too cold. Ingrid Bergman says so...


In [15]:
df_labels["annotations"][0]

('New york is lovely, Milan is nice, but london is amazing!',
 {'entities': [(0, 8, 'GPE'), (20, 25, 'GPE'), (39, 45, 'GPE')]})

### Annotation with model and EntityRuler

In [16]:
patterns = [
    {"label": "GPE", "pattern": "london"}, # this one isn't picked up by "ner"
    {"label": "GPE", "pattern": "Stockholm"},
    {"label": "PERSON", "pattern": "Humphrey Bogart"},
]

In [17]:
ruler = nlp.add_pipe("entity_ruler", config={"phrase_matcher_attr": "LOWER"}, before="ner")
ruler.add_patterns(patterns)

In [18]:
annotator = spa.Annotator(labels=["GPE", "PERSON"], model=nlp)

In [19]:
df_labels = annotator.annotate(df=df, col_text="text", shuffle=True)

HTML(value='-1 examples annotated, 3 examples left')

Text(value='', description='GPE', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='PERSON', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Output()

In [20]:
df_labels

,text,index,annotations
0,"New york is lovely, Milan is nice, but london ...",0,
1,Stockholm is too cold. Ingrid Bergman says so.,1,
